In [1]:
import numpy as np
import pandas as pd

def stock_tree(S, K, T, r, v, N, q=0, s="S"):
    deltaT = T / N
    u = np.exp(v * np.sqrt(deltaT))
    d = 1 / u
    p = (np.exp((r - q)* deltaT) - d) / (u - d)
    
    # Stock tree
    
    tree  = np.zeros((N + 1, N + 1), dtype = float)
    for i in range(0, N + 1):
        for j in range(0, i + 1):
            tree[i, j] = (S * u ** (j) * d ** (i - j))
    
    f_tree = np.zeros((N + 1, N + 1), dtype = float)
    f_tree[N,] = tree[N,]
    for i in range(N - 1, -1, -1): # Recursively calculate Future value at each node (except final).
            for j in range(0, i + 1):
                f_tree[i, j] = (f_tree[i + 1, j + 1] * p + f_tree[i + 1, j] * (1 - p))
    
    if s == 'S':
        return tree, deltaT, u, d, p, S, K, T, r, v, N, q
    else:
        return f_tree, deltaT, u, d, p, S, K, T, r, v, N, q
    
def new_bopm(option_period, model, optype='c', extype='e'):
    excercise_comparison = []
    tree, deltaT, u, d, p, S, K, T, r, v, N, q = model
    exC = np.maximum((tree - K), 0)

    exP = np.maximum((K - tree), 0)
    exC = exC[:option_period + 1, :option_period + 1]
    exP = exP[:option_period + 1, :option_period + 1]
    opTree = np.zeros((option_period + 1, option_period + 1), dtype = float)
    
    if optype == 'c':
        opTree[option_period,] = exC[option_period,] # Fix option value at final node (already  computed in the exercise tree).
    else:
        opTree[option_period,] = exP[option_period,]
    if extype == 'e':
        for i in range(option_period - 1, -1, -1): # Recursively calculate option value at each node (except final).
            for j in range(0, i + 1):
                opTree[i, j] = (opTree[i + 1, j + 1] * p + opTree[i + 1, j] * (1 - p)) / np.exp(r * deltaT)

        delta = (opTree[1, 1] - opTree[1, 0]) / (tree [1, 1] - tree[1, 0])
        
        return opTree, p, delta, opTree[0, 0], ''

    else:
        for i in range(option_period - 1, -1, -1):
            for j in range(0, i + 1):
                if (optype == 'c'): exPayoff = exC[i, j]
                else : exPayoff = exP[i, j]
                holdPayoff = (opTree[i + 1, j + 1] * p + opTree[i + 1, j] * (1 - p)) / np.exp(r * deltaT)
                if exPayoff > holdPayoff:
                    excercise_comparison.append([exPayoff, holdPayoff, i, j])
                opTree[i, j] = np.maximum(exPayoff, holdPayoff)

        delta = (opTree[1, 1] - opTree[1, 0]) / (tree [1, 1] - tree[1, 0])
#         print(excercise_comparison)
        if excercise_comparison:
            return opTree, p, delta, opTree[0, 0], excercise_comparison[-1]
        else:
            return opTree, p, delta, opTree[0, 0], ' '    
    


    
def bopm(S, K, T, r, v, N, optype='c', q=0, extype='e', s="S"):
    excercise_comparison = []
#     deltaT = T / N
#     u = np.exp(v * np.sqrt(deltaT))
#     d = 1 / u
#     p = (np.exp((r - q)* deltaT) - d) / (u - d)
    
    # Stock tree
    if s == "S":
        tree, deltaT, u, d, p, S, K, T, r, v, N, q = stock_tree(S, K, T, r, v, N, q)
    else:
        tree, deltaT, u, d, p, S, K, T, r, v, N, q = stock_tree(S, K, T, r, v, N, q, s="F")
    
#     print(tree)
    
    exC = np.maximum((tree - K), 0)

    exP = np.maximum((K - tree), 0)
        
    opTree = np.zeros((N + 1, N + 1), dtype = float)
    if optype == 'c':
        opTree[N,] = exC[N,] # Fix option value at final node (already  computed in the exercise tree).
    else:
        opTree[N,] = exP[N,]
    
    if extype == 'e':
        for i in range(N - 1, -1, -1): # Recursively calculate option value at each node (except final).
            for j in range(0, i + 1):
                opTree[i, j] = (opTree[i + 1, j + 1] * p + opTree[i + 1, j] * (1 - p)) / np.exp(r * deltaT)

        delta = (opTree[1, 1] - opTree[1, 0]) / (tree [1, 1] - tree[1, 0])
        
        return opTree, p, delta, opTree[0, 0], ''

    else:
        for i in range(N - 1, -1, -1):
            for j in range(0, i + 1):
                if (optype == 'c'): exPayoff = exC[i, j]
                else : exPayoff = exP[i, j]
                holdPayoff = (opTree[i + 1, j + 1] * p + opTree[i + 1, j] * (1 - p)) / np.exp(r * deltaT)
                if exPayoff > holdPayoff:
                    excercise_comparison.append([exPayoff, holdPayoff, i, j])
                opTree[i, j] = np.maximum(exPayoff, holdPayoff)

        delta = (opTree[1, 1] - opTree[1, 0]) / (tree [1, 1] - tree[1, 0])
#         print(excercise_comparison)
        if excercise_comparison:
            return opTree, p, delta, opTree[0, 0], excercise_comparison[-1]
        else:
            return opTree, p, delta, opTree[0, 0], ' '
        

### Question 1 

Background: You should build a 15-period binomial model whose parameters are calibrated to a Black-Scholes geometric Brownian motion model with: $T = 0.25 years$, $S_0 = 100$, $r = 2\%$, $\sigma = 30\%$ and a dividend yield of $c = 1\%$. Your binomial Model should use  u=1.0395.... (This has been rounded to four decimal places but you should not do any rounding in your spreadsheet calculations.)

Compute the price of an American call option with strike $K = 110$ and Maturity $T = 0.25 years$

Round all your answers to 2 decimal places. So if you compute a price of 12.9876 you should submit an answer of 12.99.

In [2]:
c_a_option_tree, c_a_probablity, c_a_delta, c_a_option_price, c_a_exc_comp = bopm(S=100, K=110, T=0.25, r=0.02, v=0.3, N=15, optype='c', q=0.01, extype='a')    
np.round(c_a_option_price, 3)

2.604

### Question 2

Following the background in Question 1

Compute the price of an American put option with strike $K=110$ and maturity $T = 0.25 years$.

Round all your answers to 2 decimal places. So if you compute a price of 12.9876 you should submit an answer of 12.99.

In [3]:
p_a_option_tree, p_a_probablity, p_a_delta, p_a_option_price, p_a_exc_comp = bopm(S=100, K=110, T=0.25, r=0.02, v=0.3, N=15, optype='p', q=0.01, extype='a')    
print(f"The American Put Option price is {np.round(p_a_option_price, 3)}")


The American Put Option price is 12.36


### Question 3 

Following the background in Question 1

Is it ever optimal to early exercise the put option of Question 2?

Round all your answers to 2 decimal places. So if you compute a price of 12.9876 you should submit an answer of 12.99.

In [4]:
p_e_option_tree, p_e_probablity, p_e_delta, p_e_option_price, p_e_exc_comp = bopm(S=100, K=110, T=0.25, r=0.02, v=0.3, N=15, optype='p', q=0.01, extype='e')    
if p_a_exc_comp:
    print("Yes", p_a_exc_comp)
else:
    print("No")

Yes [27.60530789182259, 27.58237863996562, 5, 0]


### Question 4

Following the background in Question 1

If your answer to Question 3 is "Yes", when is the earliest period at which it might be optimal to early exercise? (If your answer to Question 3 is "No", then you should submit an answer of 15 since exercising after 15 periods is not an early exercise.)

Round all your answers to 2 decimal places. So if you compute a price of 12.9876 you should submit an answer of 12.99.


In [5]:
print(f"The earliest to excercise is {p_a_exc_comp[2:]}")


The earliest to excercise is [5, 0]


### Question 5 

Following the background in Question 1

Do the call and put option prices of Questions 1 and 2 satisfy put-call parity?

Round all your answers to 2 decimal places. So if you compute a price of 12.9876 you should submit an answer of 12.99.

In [6]:
print(f" The put call parity holds for European style options")

 The put call parity holds for European style options


### Question 6

Following the background in Question 1

Compute the fair value of an American call option with strike $K=110$and maturity $n=10$ periods where the option is written on a futures contract that expires after 15 periods. The futures contract is on the same underlying security of the previous questions.

Round all your answers to 2 decimal places. So if you compute a price of 12.9876 you should submit an answer of 12.99.

In [7]:
t_list =stock_tree(S=100, K=110, T=0.25, r=0.02, v=0.3, N=15, q=0.01, s="F")   

f_c_a_option_tree, f_c_a_probablity, f_c_a_delta, f_c_a_option_price, f_c_a_exc_comp = new_bopm(option_period=10, model=t_list, optype='c', extype='a')
print(f"The American Call Option on Futures is {np.round(f_c_a_option_price, 2)}")

The American Call Option on Futures is 1.66


### Question 7

Following the background in Question 1

What is the earliest time period in which you might want to exercise the American futures option of Question 6?

Round all your answers to 2 decimal places. So if you compute a price of 12.9876 you should submit an answer of 12.99.

In [8]:
print(f"The earliest to excercise is {f_c_a_exc_comp[2:]}")

The earliest to excercise is [7, 7]


In [9]:
s=1
u = 2
d = 1/2
r = 0.2
k = 1

